In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import  array_to_img, load_img, img_to_array
from keras import callbacks
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.optimizers import adam
from keras.models import load_model
import cv2
import os
import gc


from numpy.random import seed
seed(2019)
#from tensorflow import set_random_seed
#set_random_seed(2019)

Using TensorFlow backend.


In [2]:
path = r"../input/images/task_images"

train_files = pd.read_csv(r'../input/convertcsv.csv') 


#convertcsv = pd.read_csv("../input/convertcsv.csv")

In [3]:
train_files.replace(['product'],1,inplace = True)
train_files.replace(['person with product'],0,inplace = True)

In [4]:

train_files['Label'].value_counts()

1    379
0    212
Name: Label, dtype: int64

In [5]:
#train_files.replace(['product'],1,inplace = True)

In [6]:
def load_images(train):
    arr = []
    for file in tqdm(train_files['Image'].values):
        img = load_img(os.path.join(path, file), target_size=(224,224))
        img = img_to_array(img)/255. # normalize image 
        arr.append(img)

    gc.collect()
    return np.asarray(arr)

In [7]:
X_train = load_images(train_files)

100%|██████████| 591/591 [00:13<00:00, 44.99it/s]


In [8]:
X_train

array([[[[0.13333334, 0.13333334, 0.24313726],
         [0.13333334, 0.13333334, 0.24313726],
         [0.13333334, 0.13333334, 0.24313726],
         ...,
         [0.73333335, 0.59607846, 0.4862745 ],
         [0.72156864, 0.5803922 , 0.47843137],
         [0.70980394, 0.56078434, 0.47058824]],

        [[0.11372549, 0.12941177, 0.23529412],
         [0.11372549, 0.12941177, 0.23529412],
         [0.10980392, 0.1254902 , 0.23137255],
         ...,
         [0.69803923, 0.57254905, 0.48235294],
         [0.6745098 , 0.5529412 , 0.47058824],
         [0.6509804 , 0.5294118 , 0.4509804 ]],

        [[0.10980392, 0.13333334, 0.23529412],
         [0.10980392, 0.13333334, 0.23529412],
         [0.10588235, 0.12941177, 0.23137255],
         ...,
         [0.6431373 , 0.5294118 , 0.45882353],
         [0.62352943, 0.5176471 , 0.4509804 ],
         [0.5921569 , 0.4862745 , 0.41960785]],

        ...,

        [[0.18431373, 0.15294118, 0.23529412],
         [0.22352941, 0.1764706 , 0.25490198]

In [9]:
from sklearn.preprocessing import OneHotEncoder
one_hot = OneHotEncoder(dtype='int8', sparse=False)
y_train = train_files['Label']

In [10]:
X_train.shape

(591, 224, 224, 3)

In [11]:
from keras.applications.mobilenet import MobileNet
model = MobileNet(include_top=False, input_shape=(224,224,3))

17227776/17225924 [==============================] - 0s 0us/step


In [12]:

from keras.models import Model
from keras.layers import Dense,Activation, GlobalAveragePooling2D

from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

x = GlobalAveragePooling2D()(model.output)
output = Dense(1, activation='sigmoid')(x)
model.summary()
model = Model(model.inputs, output)
model.save('model.hdf5')

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [13]:
#from keras.applications.resnet50 import ResNet50
#model_1 = ResNet50(include_top=False, input_shape=(224,224,3))

In [14]:
from keras.layers import Dense,Activation

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, 
                                                    stratify=y_train,
                                                    random_state=42,
                                                    test_size=0.2)

In [17]:
from keras.preprocessing.image import ImageDataGenerator
batch_size = 8
epochs = 10


model.compile(loss='binary_crossentropy', metrics=['accuracy'],
              optimizer=adam(lr=0.0001))

datagen = ImageDataGenerator(rotation_range=45, 
                             horizontal_flip=True, 
                             width_shift_range=0.5, 
                             height_shift_range=0.5, 
                             dtype='float32')

datagen.fit(X_train, augment=True, rounds=1, seed=2019)
train_generator = datagen.flow(X_train, y_train, 
                               batch_size=batch_size, 
                               seed=2019)

cp = ModelCheckpoint('best.hdf5', monitor='val_loss', save_best_only=True)
annealer = LearningRateScheduler(lambda x: 1e-4 * 0.95 ** x)

history = model.fit_generator(generator=train_generator, 
                              steps_per_epoch=len(X_train)/batch_size, 
                              validation_data=[X_test, y_test], 
                              callbacks=[cp],
                              epochs=5)

Epoch 1/5
59/59 [==============================] - 20s 342ms/step - loss: 0.6027 - accuracy: 0.6737 - val_loss: 0.3080 - val_accuracy: 0.8824
Epoch 2/5
59/59 [==============================] - 8s 128ms/step - loss: 0.4411 - accuracy: 0.7881 - val_loss: 0.3899 - val_accuracy: 0.8319
Epoch 3/5
59/59 [==============================] - 8s 140ms/step - loss: 0.4276 - accuracy: 0.8072 - val_loss: 0.5997 - val_accuracy: 0.7059
Epoch 4/5
59/59 [==============================] - 8s 141ms/step - loss: 0.3833 - accuracy: 0.8305 - val_loss: 0.6658 - val_accuracy: 0.6975
Epoch 5/5
59/59 [==============================] - 8s 141ms/step - loss: 0.3399 - accuracy: 0.8538 - val_loss: 0.3375 - val_accuracy: 0.8487


In [18]:
y_pred = model.predict(X_test)

In [19]:
arr = 1

In [20]:
arr

1

In [21]:
y_pred

array([[0.9534    ],
       [0.9778759 ],
       [0.924115  ],
       [0.938576  ],
       [0.79044604],
       [0.96338475],
       [0.05940613],
       [0.0014171 ],
       [0.9679801 ],
       [0.19518653],
       [0.98748875],
       [0.00640833],
       [0.9401648 ],
       [0.00286812],
       [0.9872836 ],
       [0.21764651],
       [0.00594366],
       [0.37440723],
       [0.2442632 ],
       [0.9942728 ],
       [0.05626145],
       [0.11412689],
       [0.00923631],
       [0.784601  ],
       [0.6065416 ],
       [0.6526304 ],
       [0.20961049],
       [0.97311735],
       [0.43819472],
       [0.00273913],
       [0.17725685],
       [0.8685593 ],
       [0.873039  ],
       [0.86310023],
       [0.9482193 ],
       [0.00158346],
       [0.0058859 ],
       [0.859028  ],
       [0.25620055],
       [0.0022544 ],
       [0.01656896],
       [0.5008458 ],
       [0.00275049],
       [0.8058795 ],
       [0.00217405],
       [0.20813203],
       [0.0305452 ],
       [0.003

In [22]:
from termcolor import colored
#from sklearn.cross_validation import KFold

from sklearn.metrics import confusion_matrix

In [23]:

# Making the Confusion Matrix
cm = confusion_matrix(y_test, (y_pred>0.5))
print(colored('The Confusion Matrix is: ', 'red'),'\n', cm)
# Calculate the accuracy on test set
predict_accuracy_on_test_set = (cm[0,0] + cm[1,1])/(cm[0,0] + cm[1,1]+cm[1,0] + cm[0,1])
print(colored('The Accuracy on Test Set is: ', 'blue'), colored(predict_accuracy_on_test_set, 'blue'))

The Confusion Matrix is:  
 [[41  2]
 [16 60]]
The Accuracy on Test Set is:  0.8487394957983193


In [24]:
np.savetxt("label_Mobile.csv", y_pred, delimiter=",")